In [28]:

import os as os
import json as js
import collections as col

import pandas as pd

cfg_path = '/TL/deep/fhgfs/projects/pebert/thesis/projects/statediff/ruffus/log'

out_root = '/TL/deep-external01/nobackup/pebert/cloudshare/mpiinf/phd/chapter_projects/statediff/supplement'

out_tex_table = os.path.join(out_root, 'supp_table_SX_runtime.tex')

table_columns = ['step', 'command', 'cores', 'samples', 'runtime (min)', 'SCIDDO version']
table_rows = []
for cfg_file in os.listdir(cfg_path):
    if not cfg_file.endswith('.json'):
        continue
    cfg_type = cfg_file.split('_')[-2]
    fpath = os.path.join(cfg_path, cfg_file)
    config = js.load(open(fpath, 'r'))
    cpus = config['args']['workers']
    runtime = config['time_min']
    if int(runtime) == 0:
        runtime = '< 1'
    else:
        # is too roughly estimated in SCIDDO
        runtime = '< ' + str(int(runtime) + 1)
    if cfg_type == 'scan':
        if config['args']['scoring'] != ['penem'] or 'cmm18' not in config['args']['dataset']:
            continue
        num_samples = '4 v 5' if 'TISSUE_Li' in config['args']['group1'] else '2 v 2'
        num_samples = '3 v 2' if 'CELLTYPE_Mo' in config['args']['group1'] else num_samples
        row = [4, 'scan', cpus, num_samples, runtime, 'v' + config['sciddo_version']]
        table_rows.append(row)
    elif cfg_type == 'convert':
        if 'ChromHMM' not in config['args']['segformat']:
            continue
        row = [1, 'convert', cpus, '9', runtime, 'v' + config['sciddo_version']]
        table_rows.append(row)
    else:
        if 'cmm18' not in config['args']['dataset']:
            continue
        if cfg_type == 'score':
            row = [3, 'score', 1, 'n/a', runtime, 'v' + config['sciddo_version']]
            table_rows.append(row)
        elif cfg_type == 'stats':
            row = [2, 'stats', cpus, '9', runtime, 'v' + config['sciddo_version']]
            table_rows.append(row)
        else:
            raise ValueError

df = pd.DataFrame(table_rows, columns=table_columns)
df.sort_values(['step', 'runtime (min)'], inplace=True)
df.reset_index(drop=True, inplace=True)
df

to_table = df.loc[[0, 1, 2, 4, 9], ['command', 'cores', 'samples', 'runtime (min)']].copy()

to_table.to_latex(out_tex_table, bold_rows=True, column_format='rccc',
                  encoding='ascii', header=True, index=False)
